In [1]:
import os
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"] = '2'
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)


In [2]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')
tf.keras.backend.clear_session()
tf.config.optimizer.set_jit(True) # Enable XLA.

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA A100 80GB PCIe, compute capability 8.0


In [3]:
from MAESeqModule.MAESeq_utils import dataloader,get_dict, seq_data_to_onehot
import numpy as np
DICT_EXIST = True
DATASET_EXIST = False
DICT_PATH_CHAR2INT = 'dict/char2int.npy'
DICT_PATH_INT2CHAR = 'dict/int2char.npy'
DATA_PATH = 'dataset/scop_fa_represeq_lib_latest.fa'
DATA_NPY_PATH = 'dataset/min20_max300_num100k.npy'

BATCH_SIZE = 256
EPOCHS = 500


if DATASET_EXIST:
    seq_list = np.load(DATA_NPY_PATH)
else:
    seq_list, max_len = dataloader(
    file= DATA_PATH,
    len_data=35000, max_len_percintile=75)

max_len = 300 # 在这里统一一下

if DICT_EXIST:
    dict_char2int = np.load(DICT_PATH_CHAR2INT, allow_pickle = True).item()
    dict_int2char = np.load(DICT_PATH_INT2CHAR,allow_pickle = True).item()
else:
    dict_char2int, dict_int2char = get_dict(seq_list)
    np.save(DICT_PATH_CHAR2INT, dict_char2int)
    np.save(DICT_PATH_INT2CHAR, dict_int2char)

onehot_data = seq_data_to_onehot(seq_list,dict_char2int,max_len)
dimension = len(dict_char2int)

print("Length of seq is {}".format(max_len))
print("Dimension is {}".format(dimension))


Length of seq is 300
Dimension is 25


In [4]:
import numpy as np
onehot_train, onehot_val,onehot_test =np.split(onehot_data,[
        int(len(onehot_data)*0.8), 
        int(len(onehot_data)*0.95)])
print(onehot_train.shape)
print(onehot_val.shape)
print(onehot_test.shape)



(28000, 300, 25)
(5250, 300, 25)
(1750, 300, 25)


In [5]:
from MAESeqModule.MAESeq_utils import mask_onehot_matrix,evaluate_per_mask_rate,extract_history

onehot_train_mask = mask_onehot_matrix(onehot_train,0.15)
onehot_val_mask = mask_onehot_matrix(onehot_val,0.15)
# train_dataset = tf.data.Dataset.from_tensor_slices((onehot_train_mask, onehot_train))
# train_dataset = train_dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE)

# val_dataset = tf.data.Dataset.from_tensor_slices((onehot_val_mask, onehot_val))
# val_dataset = val_dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE)

In [6]:
import tensorflow as tf
lr_scheduler = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate = 1e-3,
    decay_steps = int(2 * len(onehot_train)/BATCH_SIZE),
    decay_rate = 0.98,
    staircase=True
)

In [7]:
from keras import losses,optimizers,backend
from  MAESeqModule.MAESeq_model  import AutoencoderGRU_withMaskLoss,my_loss_entropy,ReconstructRateVaried
from MAESeqModule.MAESeq_utils import mask_onehot_matrix,evaluate_per_mask_rate,extract_history


autoencoder = AutoencoderGRU_withMaskLoss(latent_dim=512, encoder_shapes=(max_len, dimension))

MP_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_scheduler)
MP_optimizer = mixed_precision.LossScaleOptimizer(MP_optimizer)
          
autoencoder.compile(optimizer=MP_optimizer, 
                    loss = my_loss_entropy, 
                    metrics = [ReconstructRateVaried])
    
history = autoencoder.fit(onehot_train_mask,onehot_train,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    shuffle=True,
                    validation_data= [onehot_val_mask,onehot_val])
hist_data = extract_history(history)
hist_data.to_csv('training_history.csv')

2023-03-10 09:53:20.410936: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 09:53:21.035069: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78949 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:48:00.0, compute capability: 8.0
2023-03-10 09:53:37.434535: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1657] (One-time warning): Not using XLA:CPU for cluster.

If you want XLA:CPU, do one of the following:

 - set the TF_XLA_FLAGS to include "--tf_xla_cpu_global_jit", or
 - set cpu_global_jit to true on this session's OptimizerOptions, or
 - use experimental_jit_scope, or
 - use tf.function(jit_compile=Tr

Epoch 1/500


2023-03-10 09:53:46.215731: I tensorflow/compiler/xla/service/service.cc:171] XLA service 0x7fc410007bd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-10 09:53:46.215903: I tensorflow/compiler/xla/service/service.cc:179]   StreamExecutor device (0): NVIDIA A100 80GB PCIe, Compute Capability 8.0
2023-03-10 09:53:46.333850: I tensorflow/compiler/jit/xla_compilation_cache.cc:351] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2023-03-10 09:53:48.716581: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8100
2023-03-10 09:53:48.874071: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:237] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


110/110 [==============================] - 83s 510ms/step - loss: 2.9441 - ReconstructRateVaried: 0.8653 - val_loss: 2.8917 - val_ReconstructRateVaried: 0.8663
Epoch 2/500
110/110 [==============================] - 100s 914ms/step - loss: 2.8876 - ReconstructRateVaried: 0.8662 - val_loss: 2.8825 - val_ReconstructRateVaried: 0.8666
Epoch 3/500
110/110 [==============================] - 37s 335ms/step - loss: 2.8840 - ReconstructRateVaried: 0.8664 - val_loss: 2.8810 - val_ReconstructRateVaried: 0.8665
Epoch 4/500
110/110 [==============================] - 38s 344ms/step - loss: 2.8819 - ReconstructRateVaried: 0.8666 - val_loss: 2.8781 - val_ReconstructRateVaried: 0.8668
Epoch 5/500
110/110 [==============================] - 38s 345ms/step - loss: 2.8806 - ReconstructRateVaried: 0.8666 - val_loss: 2.8796 - val_ReconstructRateVaried: 0.8666
Epoch 6/500
106/110 [===========================>..] - ETA: 1s - loss: 2.8793 - ReconstructRateVaried: 0.8667

KeyboardInterrupt: 